In [1]:
import pandas as pd
import numpy as np
from functools import reduce
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from matplotlib import pyplot as plt
from tqdm import tqdm
import pathlib
import datetime
import dateutil

In [2]:
path = '../samples/report1.xlsx'
initial_df = pd.read_excel(path, index_col=0)

In [3]:
first_value_index = pd.to_datetime(initial_df.iloc[:, 0], 'coerce', dayfirst=True, infer_datetime_format=True).first_valid_index() 
df = initial_df.copy()[first_value_index:]
df.index = pd.to_datetime(df.index, 'coerce', dayfirst=True)

C:\Users\riabt\AppData\Local\Temp\ipykernel_21076\1223391092.py:1: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  first_value_index = pd.to_datetime(initial_df.iloc[:, 0], 'coerce', dayfirst=True, infer_datetime_format=True).first_valid_index()
C:\Users\riabt\AppData\Local\Temp\ipykernel_21076\1223391092.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  first_value_index = pd.to_datetime(initial_df.iloc[:, 0], 'coerce', dayfirst=True, infer_datetime_format=True).first_valid_index()


In [4]:
broken = '2024-05-25 11:59:59'
df.iloc[df.index.get_loc(broken) + 2]

Unnamed: 1     NaN
Unnamed: 2     NaN
Unnamed: 3     NaN
Unnamed: 4     NaN
Unnamed: 5     NaN
Unnamed: 6     NaN
Unnamed: 7     NaN
Unnamed: 8     NaN
Unnamed: 9     NaN
Unnamed: 10    NaN
Unnamed: 11    NaN
Unnamed: 12    NaN
Unnamed: 13    NaN
Unnamed: 14    NaN
Unnamed: 15    NaN
Unnamed: 16    NaN
Unnamed: 17    NaN
Unnamed: 18    NaN
Unnamed: 19    NaN
Unnamed: 20    NaN
Unnamed: 21    NaN
Unnamed: 22    NaN
Unnamed: 23    NaN
Unnamed: 24    NaN
Unnamed: 25    NaN
Unnamed: 26    NaN
Unnamed: 27    NaN
Unnamed: 28    NaN
Name: 2024-05-25 14:00:01, dtype: object

In [5]:
def load_data(initial_df):
    first_value_index = pd.to_datetime(initial_df.iloc[:, 0], 'coerce', dayfirst=True).first_valid_index() 
    df = initial_df[first_value_index:]
    df.index = pd.to_datetime(df.index, 'coerce', dayfirst=True)
    df = df.sort_index()
    data_groups = {}
    current_group = initial_df.iloc[0].values[0]

    for index, value in zip(initial_df.iloc[0].index, initial_df.iloc[0].values):
        if (pd.isna(value)):
            data_groups[current_group].append(index)
        else:
            current_group = value
            data_groups[current_group] = [index]

    data_dfs = []

    for group, cols in tqdm(zip(list(data_groups.keys()), list(data_groups.values()))):
        data_df = df[cols].copy()
        data_df.columns = initial_df[cols].iloc[1].values
        data_df.index.names = ['timestamp']
        data_dfs.append((group, data_df))

    return data_groups, data_dfs

data_groups,data_dfs = load_data(initial_df)

C:\Users\riabt\AppData\Local\Temp\ipykernel_21076\1644362668.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  first_value_index = pd.to_datetime(initial_df.iloc[:, 0], 'coerce', dayfirst=True).first_valid_index()
10it [00:00, 1111.07it/s]


In [6]:
broken = '2024-05-25 11:59:59'

In [7]:
df.iloc[df.index.get_loc(broken)  + 3]
data_dfs[0][1].iloc[data_dfs[0][1].index.get_loc(broken) + 1]

Итого       NaN
Прямое      NaN
Обратное    NaN
Name: 2024-05-25 13:00:00, dtype: object

In [ ]:
data_dfs[0][1]

,Итого,Прямое,Обратное
timestamp,,,
2023-12-01 00:59:59,367,178,189
2023-12-01 01:59:59,273,125,148
2023-12-01 02:59:59,217,106,111
2023-12-01 03:59:59,190,86,104
2023-12-01 04:59:59,242,118,124
...,...,...,...
2025-01-31 22:59:59,678,362,316
2025-01-31 23:59:59,478,246,232
NaT,12110842,NaN,NaN


In [ ]:
# data = data_dfs[0][1]['Итого'].astype(float)

# start = pd.Timestamp('2024-06-01 22:00:00')
# end = pd.Timestamp('2024-06-01 22:00:00')

# i_before = data.index.get_indexer([start]) - 1
# i_after = data.index.get_indexer([end]) + 1

# data.iloc[i_before] = np.nan
# data.iloc[i_after] = np.nan

# interpolation = data[start - pd.Timedelta(hours=12):end + pd.Timedelta(hours=12)
#      ].interpolate(method='spline', order=2)

# prediction_index =  data.index[i_before[0]:i_after[0] + 1]
# data_dfs[0][1][prediction_index, 'predict'] = interpolation[prediction_index]

In [ ]:
# data[start - pd.Timedelta(hours=12):end + pd.Timedelta(hours=12)].plot()

In [ ]:
# data_dfs[0][1]['Итого'].min()

In [19]:
def predict(df, title):
    predicted_df = pd.DataFrame(index=df.index)
    for col in df.columns:
        data = df[col].to_frame().sort_index()
        data['is_nan'] = data[col].isna().astype(int)
        data['group'] = (data['is_nan'].diff() == 1).cumsum()
        data = data.reset_index()
        nan_intervals = data[data['is_nan'] == 1].groupby(
            'group').agg(start=('timestamp', 'first'), end=('timestamp', 'last'))
        data = data.set_index('timestamp')
        data = data.drop(['is_nan', 'group'], axis=1)
        data = data.astype(float)
        nan_intervals['duration'] = nan_intervals['end'] - nan_intervals['start']
        if (len(nan_intervals) == 0):
            predicted_df[f'{title} - {col} - predict'] = pd.Series(
                    index=predicted_df.index)
            continue
        for interval in nan_intervals.itertuples():
            if (len(interval) == 0):
                predicted_df[f'{col} - predict'] = pd.Series(
                    index=predicted_df.index)
                continue
            start, end, duration = interval.start, interval.end, interval.duration
            
            if (duration < pd.Timedelta(hours=5)):
                i_before = data.index.get_loc(start) - 1
                i_after = data.index.get_loc(end) + 1
                data.iloc[i_before] = np.nan
                data.iloc[i_after] = np.nan
                interpolation = data.iloc[i_before - 12:i_after].interpolate(method='spline', order=2)
                # predict_index =  data.index[i_before[0]:i_after[0] + 1]
                predict_index = data.index[int(i_before):int(i_after) + 1]
                predicted_df[f'{title} - {col}: predict'] = pd.Series(index=data.index)
                print(predicted_df[f'{title} - {col}: predict'].loc[predict_index])
                # predicted_df[f'{title} - {col}: predict'].loc[predict_index] = interpolation[predict_index]
            else:
                train_end = data[:start].index[-2]
                train_start = (
                    train_end - pd.Timedelta(days=train_end.weekday())).normalize()
                train_start = data.iloc[data.index.get_indexer(
                    [train_start], method='nearest')[0]].name
                train = data[train_start:train_end]
                seasonal_periods = 24
                eps = 10e-05
                if len(train) < seasonal_periods * 2:
                    train = pd.concat(
                        [train, data[:train_start][len(train) - (seasonal_periods*2+1):-1]]).sort_index()
                train = np.log(train.replace(0,1) + eps)
                fit = ExponentialSmoothing(
                    train.values,
                    trend=None,
                    seasonal='mul',
                    seasonal_periods=seasonal_periods,
                    damped_trend=False,
                ).fit()
                predict_index = data.index[data.index.get_loc(
                    start-1):data.index.get_loc(end) + 2]
                nan_counts = len(data[start:end])
                data[train_start:train_end] = data[train_start:train_end] + eps
                predicted_df.loc[predict_index,
                                     f'{title} - {col}: predict'] = np.exp(fit.forecast(int(nan_counts + 1)))
                
    return predicted_df


resulted_df = pd.DataFrame(index=data_dfs[0][1].index)
for name, initial_df in data_dfs:
    predicted = predict(initial_df, name)
    resulted_df = pd.concat([resulted_df, predicted], axis=1)

timestamp
2024-05-25 11:59:59   NaN
2024-05-25 13:00:00   NaN
2024-05-25 14:00:01   NaN
2024-05-25 14:59:59   NaN
Name: Общая интенсивность автомобилей - Итого: predict, dtype: float64
timestamp
2024-06-01 20:59:59   NaN
2024-06-01 22:00:00   NaN
2024-06-01 22:59:59   NaN
Name: Общая интенсивность автомобилей - Итого: predict, dtype: float64
timestamp
2024-06-03 10:59:59   NaN
2024-06-03 12:00:00   NaN
2024-06-03 13:00:01   NaN
2024-06-03 13:59:59   NaN
Name: Общая интенсивность автомобилей - Итого: predict, dtype: float64
timestamp
2024-06-09 15:59:59   NaN
2024-06-09 17:00:00   NaN
2024-06-09 18:00:01   NaN
2024-06-09 18:59:59   NaN
Name: Общая интенсивность автомобилей - Итого: predict, dtype: float64
timestamp
2024-06-15 15:59:59   NaN
2024-06-15 17:00:00   NaN
2024-06-15 17:59:59   NaN
Name: Общая интенсивность автомобилей - Итого: predict, dtype: float64
timestamp
2024-07-19 11:59:59   NaN
2024-07-19 13:00:00   NaN
2024-07-19 13:59:59   NaN
Name: Общая интенсивность автомобилей -

d:\projects\PycharmProjects\transportation\venv\lib\site-packages\pandas\core\missing.py:604: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  terp = interpolate.UnivariateSpline(x, y, k=order, **kwargs)
d:\projects\PycharmProjects\transportation\venv\lib\site-packages\pandas\core\missing.py:604: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  terp = interpolate.UnivariateSpline(x, y, k=order, **kwargs)


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [12]:
data_dfs[0][1][pd.to_datetime(['2024-05-24 19:59:59', '2024-05-24 20:59:59','2024-05-24 21:59:59', '2024-05-24 22:59:59']), 'new']

InvalidIndexError: (DatetimeIndex(['2024-05-24 19:59:59', '2024-05-24 20:59:59',
               '2024-05-24 21:59:59', '2024-05-24 22:59:59'],
              dtype='datetime64[ns]', freq=None), 'new')

In [ ]:
data_dfs[0][1]['Обратное'] + 10e-05

timestamp
2024-01-01 00:59:59     65.0001
2024-01-01 01:59:59    134.0001
2024-01-01 02:59:59    106.0001
2024-01-01 03:59:59     73.0001
2024-01-01 04:59:59     42.0001
                         ...   
2024-12-31 19:59:59    284.0001
2024-12-31 20:59:59    212.0001
2024-12-31 21:59:59    186.0001
2024-12-31 22:59:59    130.0001
2024-12-31 23:59:59     47.0001
Name: Обратное, Length: 8771, dtype: object

In [ ]:
for name, initial_df in data_dfs:
    print(name)

Общая интенсивность автомобилей
Легковые (до 6 м)
Малые груз. (6-9 м)
Грузовые (9-13 м)
Груз. большие (13-22 м)
Автопоезда (22-30 м)
Автобусы
Мотоциклы
Скорость, км/ч
Загрузка, %


In [ ]:
# = pd.DataFrame(index=data_dfs[0][1].index)
# resulted_df['Общая интенсивность автомобилей - Итого: predict'][~resulted_df['Общая интенсивность автомобилей - Итого: predict'].isna()]
# for name, df in data_dfs:
#     resulted_df = pd.concat([resulted_df, df], axis=1)
    

In [ ]:
info_row = [np.nan]
for col_name, cols in zip(list(data_groups.keys())[1:], list(data_groups.values())[1:]):
    info_row += [col_name] * len(cols)

In [ ]:
list(zip(info_row, resulted_df.columns))

[(nan, 'Общая интенсивность автомобилей - Итого - predict'),
 ('Общая интенсивность автомобилей',
  'Общая интенсивность автомобилей - Прямое - predict'),
 ('Общая интенсивность автомобилей',
  'Общая интенсивность автомобилей - Обратное: predict'),
 ('Общая интенсивность автомобилей', 'Легковые (до 6 м) - Итого - predict'),
 ('Легковые (до 6 м)', 'Легковые (до 6 м) - Прямое - predict'),
 ('Легковые (до 6 м)', 'Легковые (до 6 м) - Обратное: predict'),
 ('Легковые (до 6 м)', 'Малые груз. (6-9 м) - Итого - predict'),
 ('Малые груз. (6-9 м)', 'Малые груз. (6-9 м) - Прямое - predict'),
 ('Малые груз. (6-9 м)', 'Малые груз. (6-9 м) - Обратное: predict'),
 ('Малые груз. (6-9 м)', 'Грузовые (9-13 м) - Итого - predict'),
 ('Грузовые (9-13 м)', 'Грузовые (9-13 м) - Прямое - predict'),
 ('Грузовые (9-13 м)', 'Грузовые (9-13 м) - Обратное: predict'),
 ('Грузовые (9-13 м)', 'Груз. большие (13-22 м) - Итого - predict'),
 ('Груз. большие (13-22 м)', 'Груз. большие (13-22 м) - Прямое - predict'),
 ('

In [ ]:
len(info_row), resulted_df.shape

(29, (8771, 28))

In [ ]:
resulted_df = resulted_df.reset_index()
resulted_df.loc[-1] = info_row

C:\Users\riabt\AppData\Local\Temp\ipykernel_24540\4257859202.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  resulted_df.loc[-1] = info_row


In [ ]:
resulted_df.index = resulted_df.index + 1
resulted_df = resulted_df.sort_index()

In [ ]:
save_path = str(f'{pathlib.Path(path).parent}/output/{title.replace("/", "-").replace(" ", "_")}__{datetime.datetime.now().strftime("%d-%m-%Y__%H-%M-%S")}.xslx')
save_path

'..\\samples/output/км_32+000_а-д_А-146_Краснодар-Верхнебаканский___31-03-2025__00-51-51.xslx'

In [ ]:
initial_df

NameError: name 'initial_df' is not defined

In [ ]:
with pd.ExcelWriter(path,  engine='xlsxwriter') as writer:
    # исходные данные
    initial_df.to_excel(excel_writer=writer, 
                         sheet_name='Исходные данные', index=False)
    # предсказанные данные
    resulted_df = resulted_df.reset_index().rename(columns={'timestamp': 'Дата'})
    if 'level_0' in resulted_df.columns:
        resulted_df.drop('level_0', axis=1, inplace=True)
    if 'index' in resulted_df.columns:
        resulted_df.drop('index', axis=1, inplace=True)
    resulted_df['Дата'] = resulted_df['Дата'].astype(str)
    resulted_df.to_excel(excel_writer=writer, 
                         sheet_name='Результаты', index=False)